In [20]:
import json
import numpy as np
import os
import zipfile
import math

from builelib.make_inputdata import (
    make_jsondata_from_Ver2_sheet,
    make_jsondata_from_Ver4_sheet,
)
from builelib import (
    airconditioning_webpro,
    ventilation,
    lighting,
    hotwatersupply,
    elevator,
    photovoltaic,
    other_energy,
    cogeneration,
)

# json.dump用のクラスは今は省いている
inputfile_name = "./sample/WEBPRO_inputSheet_sample.xlsm"  # 入力ファイル指定
inputdata, validation = make_jsondata_from_Ver2_sheet(inputfile_name)

In [21]:

# データ定義セクション
calc_reuslt = {
    "設計一次エネルギー消費量[MJ]": 0,
    "基準一次エネルギー消費量[MJ]": 0,
    "設計一次エネルギー消費量（その他除き）[MJ]": 0,
    "基準一次エネルギー消費量（その他除き）[MJ]": 0,
    "BEI": "",
    "設計一次エネルギー消費量（再エネ、その他除き）[MJ]": 0,
    "BEI（再エネ除き）": "",
    "設計一次エネルギー消費量（空調）[MJ]": 0,
    "基準一次エネルギー消費量（空調）[MJ]": 0,
    "BEI_AC": "-",  # BEI（空調）
    "設計一次エネルギー消費量（換気）[MJ]": 0,
    "基準一次エネルギー消費量（換気）[MJ]": 0,
    "BEI_V": "-",  # BEI（換気）
    "設計一次エネルギー消費量（照明）[MJ]": 0,
    "基準一次エネルギー消費量（照明）[MJ]": 0,
    "BEI_L": "-",  # BEI（照明）
    "設計一次エネルギー消費量（給湯）[MJ]": 0,
    "基準一次エネルギー消費量（給湯）[MJ]": 0,
    "BEI_HW": "-",  # BEI（給湯）
    "設計一次エネルギー消費量（昇降機）[MJ]": 0,
    "基準一次エネルギー消費量（昇降機）[MJ]": 0,
    "BEI_EV": "-",  # BEI（昇降機）
    "その他一次エネルギー消費量[MJ]": 0,
    "創エネルギー量（太陽光）[MJ]": 0,
    "創エネルギー量（コジェネ）[MJ]": 0,
}

# CGSの計算に必要となる変数
resultJson_for_CGS = {
    "AC": {},
    "V": {},
    "L": {},
    "HW": {},
    "EV": {},
    "PV": {},
    "OT": {},
}

# 設計一次エネルギー消費量[MJ]
energy_consumption_design = 0
# 基準一次エネルギー消費量[MJ]
energy_consumption_standard = 0

# 結果いれるリスト
resultdata_AC = {}
resultdata_V = {}


In [22]:
# AC計算セクション
resultdata_AC = airconditioning_webpro.calc_energy(inputdata, debug = False)

# CGSの計算に必要となる変数
resultJson_for_CGS["AC"] = resultdata_AC["for_CGS"]

# 設計一次エネ・基準一次エネに追加
energy_consumption_design += resultdata_AC["設計一次エネルギー消費量[MJ/年]"]
energy_consumption_standard += resultdata_AC["基準一次エネルギー消費量[MJ/年]"]
calc_reuslt["設計一次エネルギー消費量（空調）[MJ]"] = resultdata_AC[
    "設計一次エネルギー消費量[MJ/年]"
]
calc_reuslt["基準一次エネルギー消費量（空調）[MJ]"] = resultdata_AC[
    "基準一次エネルギー消費量[MJ/年]"
]

室負荷計算完了
空調負荷計算完了
空調機群のエネルギー消費量計算完了
ポンプ負荷計算完了
二次ポンプ群のエネルギー消費量計算完了
熱源負荷計算完了
熱源エネルギー計算完了


In [23]:
# 機械換気計算セクション---------------------------------------------------------
resultdata_V = ventilation.calc_energy(inputdata, DEBUG = False)
# CGSの計算に必要となる変数
resultJson_for_CGS["V"] = resultdata_V["for_CGS"]
# 設計一次エネ・基準一次エネに追加
energy_consumption_design += resultdata_V["設計一次エネルギー消費量[MJ/年]"]
energy_consumption_standard += resultdata_V["基準一次エネルギー消費量[MJ/年]"]
calc_reuslt["設計一次エネルギー消費量（換気）[MJ]"] = resultdata_V["設計一次エネルギー消費量[MJ/年]"]
calc_reuslt["基準一次エネルギー消費量（換気）[MJ]"] = resultdata_V["基準一次エネルギー消費量[MJ/年]"]
calc_reuslt["BEI_V"] = math.ceil(resultdata_V["BEI/V"] * 100) / 100